In [1]:
pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import fitsio

In [3]:
i3_fits=fitsio.FITS("/global/cscratch1/sd/troxel/finaly1cats/y1a1-im3shape_v5_matched_v6.fits")
mc_fits=fitsio.FITS("/global/cscratch1/sd/troxel/finaly1cats/mcal-y1a1-combined-riz-blind-v4-matched.fits")

In [29]:
mc_data=mc_fits[1].read_columns(['snr', 'R11', 'R22', 'flags_select', 'e1', 'covmat_0_0'])

In [5]:
i3_data=i3_fits[1].read_columns(['snr', 'weight', 'flags_select', 'e1', 'm'])

In [6]:
i3_select=i3_data['flags_select']==0
i3_snr = i3_data['snr'][i3_select]
i3_w = i3_data['weight'][i3_select]
i3_e1 = i3_data['e1'][i3_select]

In [30]:
mc_select=mc_data['flags_select']==0
mc_snr = mc_data['snr'][mc_select]
mc_R =   0.5*(mc_data['R11'][mc_select]+mc_data['R22'][mc_select])
mc_e1 =  mc_data['e1'][mc_select]
mc_C00 = mc_data['covmat_0_0'][mc_select]

In [8]:
area = 1515. * 60 * 60

In [33]:
# Im3shape raw
i3_raw = i3_select.sum() / area
# Metacal raw
mc_raw = mc_select.sum() / area

# Im3shape heymans
i3_hey = i3_w.sum()**2 / (i3_w**2).sum() / area
#Metacal heymans
mc_hey = mc_raw

#Im3shape Chang
i3_shape = i3_e1[i3_snr>100].var()
i3_total = 1.0/i3_w.sum()
i3_chang1 = i3_shape / i3_total / area
i3_chang2 = i3_e1[i3_snr>150].var() * i3_w.sum() / area

#Metacal Chang
mc_shape = mc_e1[mc_snr>100].std() / mc_R[mc_snr>100].mean()
mc_noise = mc_C00 / mc_R.mean()**2
mc_chang =  (mc_shape / (mc_shape+mc_noise)).sum() / area


In [34]:
print """
i3_raw: {i3_raw}
mc_raw: {mc_raw}
i3_hey: {i3_hey}
mc_hey: {mc_hey}
i3_chang: {i3_chang1}, {i3_chang2}
mc_chang: {mc_chang}
""".format(**locals())


i3_raw: 4.51806398973
mc_raw: 6.38786541988
i3_hey: 4.16058148048
mc_hey: 6.38786541988
i3_chang: 3.35045471856, 3.27105948099
mc_chang: 6.24020258654

